<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/pandas/pandas_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Clean Up Pandas Data

Here we should how to clean up Pandas data, in particular what do with about missing data, as well as what to do with invalid data.

Consider a survey.  If you send people a survey you cannot control in all cases what questions they will answer.  And you cannot anticipate what data they might put that is invalid.  For example, they might leave off some data.  Or they might enter a number in a question that is only supposed to be text.

So here we show you how to :

* generate some random data that is purposefully not clean
* drop duplicate rows
* get rid of rows that have missing values
* convert missing values to something else, like a fixed value of the average of all the other values in the colummn
* delete outliers, which are obvious typos.  For example, here we enter some salaries as 1 million while everyone else is around 100,000.  So that's most likely a mistake (since these are employees and not company owners or the CEO).
* apply a custom function to every row to do whatever special checking you want
* check for missing values
* show different ways to check for numbers or strings and how to convert those when they are of the wrong type

In [67]:
import numpy as np
import pandas as pd

import random

mean=10000
std=25



cols = [("name", str), ("income",np.int8), ("education", str),
     ("age", np.int8), ("city",str), ("id", np.int8), ("email", str), ("salary", np.int8),
        ("citizen", ["Y", "N"])]

words = [np.NaN, "", "abc", "def", "ghi", "jkl", "mno", "pqr"]

records = []


for i in range(20):

  data = {}

  for c in cols:

    if c[1] == np.int8:
      if random.randint(0,5)==5:
          data[c[0]] = np.NaN
      else:
          data[c[0]] = abs(int(random.gauss(mean, std)))

    if c[0] == "citizen":
       if random.randint(0,5)==5:
          data[c[0]] = random.randint(0,10)
       else:
          data[c[0]] =c[1][random.randint(0,1)]

    if c[1] == str and c[0] != "citizen":
       data[c[0]] = words[random.randint(0,len(words)-1)]

    if (c[0] == "income") & (random.randint(0,5)==0):
        data[c[0]] = chr(random.randint(65,70))

    if (c[0] == "income") & (random.randint(0,10)==0):
        data[c[0]] = 1000000

    if (c[0] == "age"):
        data[c[0]] = random.randint(20,65)

  records.append(data)

df=pd.DataFrame(records)


df

,name,income,education,age,city,id,email,salary,citizen
0,def,NaN,abc,35,mno,9979.0,pqr,9989.0,N
1,jkl,10050,abc,24,def,10013.0,NaN,NaN,N
2,pqr,NaN,pqr,24,jkl,9994.0,ghi,10034.0,N
3,def,9985,ghi,48,jkl,9966.0,pqr,9956.0,Y
4,abc,C,NaN,44,def,9982.0,abc,10001.0,N
5,jkl,10041,pqr,35,mno,10026.0,,9993.0,N
6,ghi,10002,pqr,40,pqr,9990.0,,9994.0,0
7,def,1000000,ghi,43,NaN,9989.0,,9998.0,Y
8,abc,9999,,59,jkl,10017.0,,10019.0,N
9,pqr,9994,jkl,58,,NaN,def,9983.0,Y


In [68]:
# check if column in series is empty

df['name'].isnull()


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
Name: name, dtype: bool

In [69]:
# draw from mean()

sMean=df['salary'].mean()

df['salary'] = df['salary'].fillna(sMean)

df

,name,income,education,age,city,id,email,salary,citizen
0,def,NaN,abc,35,mno,9979.0,pqr,9989.000000,N
1,jkl,10050,abc,24,def,10013.0,NaN,9990.944444,N
2,pqr,NaN,pqr,24,jkl,9994.0,ghi,10034.000000,N
3,def,9985,ghi,48,jkl,9966.0,pqr,9956.000000,Y
4,abc,C,NaN,44,def,9982.0,abc,10001.000000,N
5,jkl,10041,pqr,35,mno,10026.0,,9993.000000,N
6,ghi,10002,pqr,40,pqr,9990.0,,9994.000000,0
7,def,1000000,ghi,43,NaN,9989.0,,9998.000000,Y
8,abc,9999,,59,jkl,10017.0,,10019.000000,N
9,pqr,9994,jkl,58,,NaN,def,9983.000000,Y


In [70]:
# Yes, No columns cannot contain numbers
# NaN is OK as we can get rid of it in a second step plus certain functions like mean()
# will ignore it.  coerce means convert to NaN on error

df['income'] = df['income'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

df['income']

0           NaN
1       10050.0
2           NaN
3        9985.0
4           NaN
5       10041.0
6       10002.0
7     1000000.0
8        9999.0
9        9994.0
10          NaN
11          NaN
12       9983.0
13      10034.0
14       9995.0
15    1000000.0
16          NaN
17      10023.0
18       9972.0
19          NaN
Name: income, dtype: float64

In [71]:
# drop outliers, too many standard deviations away


mean = df['income'].mean()
std_dev = df['income'].std()

# Define threshold for outliers (e.g., values more than 2 standard deviations away from the mean)
threshold = 2

df.loc[(df['income'] - mean) > 2 * std_dev]



,name,income,education,age,city,id,email,salary,citizen
7,def,1000000.0,ghi,43,NaN,9989.0,,9998.0,Y
15,def,1000000.0,jkl,46,NaN,10008.0,NaN,9990.0,N


In [72]:
df.drop(df.loc[(df['income'] - mean) > 2 * std_dev].index, inplace=True)

df

,name,income,education,age,city,id,email,salary,citizen
0,def,NaN,abc,35,mno,9979.0,pqr,9989.000000,N
1,jkl,10050.0,abc,24,def,10013.0,NaN,9990.944444,N
2,pqr,NaN,pqr,24,jkl,9994.0,ghi,10034.000000,N
3,def,9985.0,ghi,48,jkl,9966.0,pqr,9956.000000,Y
4,abc,NaN,NaN,44,def,9982.0,abc,10001.000000,N
5,jkl,10041.0,pqr,35,mno,10026.0,,9993.000000,N
6,ghi,10002.0,pqr,40,pqr,9990.0,,9994.000000,0
8,abc,9999.0,,59,jkl,10017.0,,10019.000000,N
9,pqr,9994.0,jkl,58,,NaN,def,9983.000000,Y
10,abc,NaN,jkl,55,mno,9979.0,pqr,9983.000000,N


In [73]:
# drop all numbers in Yes/No answer

df['citizen'] = df['citizen'].apply(lambda x: x if isinstance(x,str) else np.NaN)
df['citizen']


0       N
1       N
2       N
3       Y
4       N
5       N
6     NaN
8       N
9       Y
10      N
11      Y
12      Y
13      Y
14    NaN
16      N
17      N
18      Y
19      Y
Name: citizen, dtype: object

In [74]:
df

,name,income,education,age,city,id,email,salary,citizen
0,def,NaN,abc,35,mno,9979.0,pqr,9989.000000,N
1,jkl,10050.0,abc,24,def,10013.0,NaN,9990.944444,N
2,pqr,NaN,pqr,24,jkl,9994.0,ghi,10034.000000,N
3,def,9985.0,ghi,48,jkl,9966.0,pqr,9956.000000,Y
4,abc,NaN,NaN,44,def,9982.0,abc,10001.000000,N
5,jkl,10041.0,pqr,35,mno,10026.0,,9993.000000,N
6,ghi,10002.0,pqr,40,pqr,9990.0,,9994.000000,NaN
8,abc,9999.0,,59,jkl,10017.0,,10019.000000,N
9,pqr,9994.0,jkl,58,,NaN,def,9983.000000,Y
10,abc,NaN,jkl,55,mno,9979.0,pqr,9983.000000,N


In [75]:
# now we are left with only those rows that have all non-Nan columns.  Note that we still
# have some blanks sells.  You would choose the various procedures we have explained above
# to determine how you want to handle this situation.

df.dropna(how='any', inplace=True)
df

,name,income,education,age,city,id,email,salary,citizen
3,def,9985.0,ghi,48,jkl,9966.0,pqr,9956.000000,Y
5,jkl,10041.0,pqr,35,mno,10026.0,,9993.000000,N
8,abc,9999.0,,59,jkl,10017.0,,10019.000000,N
12,abc,9983.0,mno,28,jkl,10031.0,abc,9997.000000,Y
17,def,10023.0,def,45,def,10005.0,def,9990.944444,N


In [76]:
# here we operate on a Pandas series, which is a single column.  But since we are
# sending each row one row at a time we are operating on a single row-column combination
# This s is a single value, a scalar.  So we can work with it as we would with any
# Python primitive (meaning built-in type.  Pandas and Numpy are extension of Python into new types.)


def toUpper(s):
    if isinstance(s,str):
      return s.upper()


df['city']=df['city'].apply(toUpper)
df

,name,income,education,age,city,id,email,salary,citizen
3,def,9985.0,ghi,48,JKL,9966.0,pqr,9956.000000,Y
5,jkl,10041.0,pqr,35,MNO,10026.0,,9993.000000,N
8,abc,9999.0,,59,JKL,10017.0,,10019.000000,N
12,abc,9983.0,mno,28,JKL,10031.0,abc,9997.000000,Y
17,def,10023.0,def,45,DEF,10005.0,def,9990.944444,N


In [77]:
# axis = 1 means row
# axis = 0 means column

# the important point to note there is we send the entire row in as a paramters. thus all the columns
# are available to use.  remember to send back the entire row after you have updated any of the columns.

def wholeRow(row):
  if isinstance(row['city'],str):
     row['city'] = row['city'].lower()
     return row


df=df.apply(wholeRow, axis=1)
df

,name,income,education,age,city,id,email,salary,citizen
3,def,9985.0,ghi,48,jkl,9966.0,pqr,9956.000000,Y
5,jkl,10041.0,pqr,35,mno,10026.0,,9993.000000,N
8,abc,9999.0,,59,jkl,10017.0,,10019.000000,N
12,abc,9983.0,mno,28,jkl,10031.0,abc,9997.000000,Y
17,def,10023.0,def,45,def,10005.0,def,9990.944444,N


In [78]:
df

,name,income,education,age,city,id,email,salary,citizen
3,def,9985.0,ghi,48,jkl,9966.0,pqr,9956.000000,Y
5,jkl,10041.0,pqr,35,mno,10026.0,,9993.000000,N
8,abc,9999.0,,59,jkl,10017.0,,10019.000000,N
12,abc,9983.0,mno,28,jkl,10031.0,abc,9997.000000,Y
17,def,10023.0,def,45,def,10005.0,def,9990.944444,N


# Eliminate Duplicates

eliminate duplicates.  the way to do that is to group all columns and sum.  If the count is bigger then one them delete all but one
